# Strands Agents with Bedrock AgentCore Code Interpreter

This workshop demonstrates how to integrate Strands Agents with Amazon Bedrock AgentCore Code Interpreter to create AI agents capable of executing code dynamically.

## Overview

In this lab, you will:
- Learn about Bedrock AgentCore Code Interpreter capabilities
- Test the default Code Interpreter with Strands Agents
- Create a custom Code Interpreter with public network access
- Compare different execution environments and their limitations

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Basic understanding of Strands Agents and Bedrock AgentCore concepts

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for Strands Agents and Bedrock AgentCore Python SDK:

In [ ]:
%pip install -q strands-agents strands-agents-tools bedrock-agentcore

## What is Bedrock AgentCore Code Interpreter?

Amazon Bedrock AgentCore Code Interpreter is a powerful tool that allows AI agents to execute code dynamically in a secure sandbox environment. Key benefits include:

- **Secure Execution**: Runs Python code in an isolated sandbox environment
- **Dynamic Problem Solving**: Enables agents to perform calculations, analyze data, and generate visualizations
- **Flexible Configuration**: Supports both default sandboxed and custom network-enabled environments
- **Integration Ready**: Seamlessly integrates with Strands Agents and other AI frameworks

The Code Interpreter provides agents with the ability to solve complex problems that require computational analysis, data processing, or mathematical calculations.

### Testing Strands Agent with Default Code Interpreter

Let's start by testing the Strands Agent using the default AgentCore Code Interpreter. We'll demonstrate how it can generate and execute code to solve mathematical problems in a sandboxed environment.

In [ ]:
from strands import Agent
from strands_tools.code_interpreter import AgentCoreCodeInterpreter

agentcore_code_interpreter = AgentCoreCodeInterpreter()

# Create a code-gen assistant agent
agent = Agent(
    model="us.amazon.nova-pro-v1:0", 
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.""",
    tools=[agentcore_code_interpreter.code_interpreter],
)

agent("Tell me the largest random prime number between 1 and 100, which is less than 84 and more that 9")

Let's examine the detailed execution flow of the agent loop to understand how the agent processes requests and generates responses:

In [ ]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

for message in agent.messages:
    for content in message['content']:
        if 'toolUse' in content and content['toolUse']['input']['code_interpreter_input']['action']['type']=='executeCode':
            code = content['toolUse']['input']['code_interpreter_input']['action']['code']
            print("\nGenerated Code")
            print("--------------")
            print(code)

### Testing Network Limitations in Default Environment

Now let's test the network capabilities of the default Code Interpreter by attempting a web search scenario. This will demonstrate the sandbox limitations:

In [ ]:
# Create a recipe assistant agent
agent = Agent(
    model="us.amazon.nova-pro-v1:0", 
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.
                  If you has no available tools to perform the task, you must generate and execute code to continue. 
                  If required, execute pip install to download required packages.
                  """,
    tools=[agentcore_code_interpreter.code_interpreter],
)

agent("What is the stock price of Amazon today?")

Let's examine the detailed execution flow for the web search scenario:

In [ ]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

for message in agent.messages:
    for content in message['content']:
        if 'toolUse' in content and content['toolUse']['input']['code_interpreter_input']['action']['type']=='executeCode':
            code = content['toolUse']['input']['code_interpreter_input']['action']['code']
            print("\nGenerated Code")
            print("--------------")
            print(code)

### Understanding Sandbox Environment Limitations

The above web search scenario demonstrates that the default Code Interpreter operates in a sandbox environment with network isolation. This is an important security feature that prevents unauthorized network access while ensuring code execution safety.

## Creating a Custom Code Interpreter with Network Access

To enable web-based operations, we'll create a custom Code Interpreter with public network access. This demonstrates the flexibility of the AgentCore platform for different use cases.

### Step 1: Initialize AgentCore Clients

First, let's set up the necessary clients for both control plane and data plane operations:

In [ ]:
from bedrock_agentcore._utils import endpoints
import boto3
import json

region = boto3.session.Session().region_name

data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

### Step 2: Create Custom Code Interpreter

Create a custom Code Interpreter with public network access:

In [ ]:
import uuid

# Create code interpreter
unique_name = f"sampleCodeInterpreter_{uuid.uuid4().hex[:8]}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="Environment for Code Interpreter sample test",
    #executionRoleArn=iam_role_arn, #Required only if the code interpreter need to access AWS resources
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Created interpreter: {interpreter_id}")

### Step 3: Create Code Interpreter Session

Create a code interpreter session in the custom Code Interpreter:

In [ ]:
# Create code interpreter session
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="sampleCodeInterpreterSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"Created session: {session_id}")

### Step 4: Test Basic Functionality

Let's test the code interpreter session by executing a simple "Hello World" command:

In [ ]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "echo 'Hello World'"
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

Let's verify if the code interpreter session can access the internet by installing the DuckDuckGo Python package with pip:

In [ ]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "pip install yfinance"
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

Let's execute Python code to use DuckDuckGo for web search with the keyword "coffee maker amazon.com" in the code interpreter session:

In [ ]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCode",
    arguments={"code": """
        import yfinance as yf
               
        amzn = yf.Ticker('AMZN')
        data = amzn.history(period='1d')
        today_close = data['Close'][-1]
        print(today_close)
        """,
        "language": "python",
        "clearContext": False
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

## Using Custom Code Interpreter with Strands Agent

Now we'll integrate the custom Code Interpreter with internet access into Strands Agent. We'll create custom `execute_python` and `execute_command` tools that share the same code interpreter session for enhanced functionality.

In [ ]:
from strands import Agent, tool
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import boto3

#Reuse the Core Interpreter and Session created above
ci_client = CodeInterpreter(region=boto3.session.Session().region_name)
ci_client.start(identifier=interpreter_id) #initializes a new code interpreter session

@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""
    
    print(f"\n Generated Code: {code}")
    response = ci_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
        
    for event in response["stream"]:
        return json.dumps(event["result"])

@tool
def execute_command(command: str, description: str = "") -> str:
    """Execute command in the sandbox."""
    
    print(f"\n Generated Command: {command}")
    response = ci_client.invoke("executeCommand", {
        "command": command
    })
    
    for event in response["stream"]:
        return json.dumps(event["result"])
    

# Create a code gen agent
agent = Agent(
    model="us.amazon.nova-pro-v1:0", 
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.
                  If you has no available tools to perform the task, you must generate and execute code to continue. 
                  If required, execute pip install to download required packages.
                  """,
    tools=[execute_python, execute_command],
)

agent("What is the stock price of Amazon today?")

ci_client.stop() #stop the current code interpreter session

Let's examine the detailed execution flow for the custom tools scenario:

In [ ]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

## Resource Cleanup (Optional)

Clean up the AgentCore Runtime resources we created to avoid unnecessary charges:

In [ ]:
import boto3

cp_client = boto3.client("bedrock-agentcore-control", region_name=region, endpoint_url=control_plane_endpoint)
dp_client = boto3.client("bedrock-agentcore", region_name=region, endpoint_url=data_plane_endpoint)

try:
    print("Cleaning up session and interpreter...")
    dp_client.stop_code_interpreter_session(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id
    )
    print("✓ Session stopped successfully")

    cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
    print("✓ Interpreter deleted successfully")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")